In [2]:
# Document loading, retrieval methods and text splitting
%pip install -qU langchain langchain_community

# Local vector store via Chroma
%pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install -qU langchain_ollama

# Web Loader
%pip install -qU beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


https://python.langchain.com/api_reference/ollama/llms/langchain_ollama.llms.OllamaLLM.html#langchain_ollama.llms.OllamaLLM.model

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
loader = PyPDFLoader("LocalDocs/MIL-STD-882E.pdf")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [5]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings


In [6]:

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [7]:
question = "What are system safety requirements?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [8]:
docs[0]

Document(id='75faf8dd-19cd-49fb-8aa8-df12ec268c0b', metadata={'page': 14, 'page_label': '15', 'source': 'LocalDocs/MIL-STD-882E.pdf'}, page_content='4.2  System safety requirements.  Section 4 defines the system safety requirements \nthroughout the life-cycle for any system.  When properly applied, these requirements should \nenable the identification and management of hazards and their associated risks during system \ndevelopmental and sustaining engineering activities. It is not the intent of this document to make \nsystem safety personnel responsible for hazard management in other functional disciplines.')

In [20]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2:1b",
    num_gpu = -1
)

In [21]:
response_message = model.invoke(
    "Simulate a rap battle between Stephen Colbert and John Oliver"
)

print(response_message.content)

**The Scene:** A dark, crowded nightclub. The air is electric with anticipation as two of the most formidable voices in comedy take to the mic. In one corner, we have Stephen Colbert, the former host of "The Daily Show" and current host of "The Late Show." Across from him, John Oliver, the charismatic and irreverent creator of "Last Week Tonight" stands tall, ready to take on the king of satire.

**Stephen Colbert:**
Yo, J-O-N, I hear you've been talking smack
About my humor, about my comedic pack
But let me tell you, I'm the one who's got the skill
I can make a joke that'll make your head spin and thrill

My wit's sharp as a razor, my sarcasm's on point
I can skewer politicians like a hot knife through a bone
You may have your fancy writing, your clever rhymes
But when it comes to real comedy, I'm the one who shines

**John Oliver:**
Hold up, Steve, let me interrupt your flow
You think you're funny, but your humor's just a show
I can make fun of you, and my audience will too
You may h

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

In [ ]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [ ]:
question = "What are the approaches to Task Decomposition?"

qa_chain.invoke(question)